In [1]:
# # Run this and then restart the kernel at the start of each session to install
# # 'teotil3' in development mode
# !pip install -e /home/jovyan/projects/teotil3/

In [2]:
import os

import geopandas as gpd
import nivapy3 as nivapy
import pandas as pd
import teotil3 as teo

In [3]:
# Connect to JupyterHub's PostGIS database
eng = nivapy.da.connect_postgis()

Connection successful.


# Invest4Nature: Discharges to the coast at municipality level

As part of Bo Stoknes' Masters thesis, we want to estimate:

 1. Inputs of TOTN from each kommune to the river network, and
    
 2. How much of the TOTN from each kommune reaches the coastline.

This notebook creates initial estimates based on results from TEOTIL3.

## 1. User options

We are interested in **TOTN** and I will calculate **average annual fluxes** from each kommune over the period from **2013 to 2023**. I will use the kommune boundaries from 2023 to link kommuner to regine units.

In [4]:
# Parameter of interest
par = "totn"

# TEOTIL options
nve_data_year = 2024
st_yr, end_yr = 2013, 2023
agri_loss_model = "annual"  # 'annual' or 'risk'
res_fold = r"/home/jovyan/shared/common/teotil3/evaluation"
core_data_fold = "/home/jovyan/shared/common/teotil3/core_data"

# Excel file to create
xl_path = r"../data/kommune_inputs_to_coast.xlsx"

## 2. Get data

In [5]:
# Read TEOTIL3 results file for period of interest
mod_csv = os.path.join(
    res_fold,
    f"teo3_results_nve{nve_data_year}_{st_yr}-{end_yr}_agri-{agri_loss_model}-loss.csv",
)
mod_df = pd.read_csv(mod_csv).query("@st_yr <= year <= @end_yr")

# Get regines linked to each kommune (use kommune boundaries for 'end_yr')
reg_gdf = teo.io.get_regine_geodataframe(eng, end_yr)[["regine", "komnr"]]

# Get transmission factors to coast for each regine
teo_gpkg = os.path.join(core_data_fold, "tidied", "teotil3_data.gpkg")
trans_df = gpd.read_file(teo_gpkg, layer="transmission_to_coast")

## 3. Estimates nutrient fluxes

The code below does the following:

 1. Estimates the amount of TOTN from each source added to each regine.
    
 2. Using transmission factors from TEOTIL3, estimates the amount of TOTN from each source in each regine that reaches the coast.

 3. Aggregates the regine-level data to kommune level.
    
 4. Calculates average fluxes per year.

The result is an Excel file with **one row per kommune** and columns named as follows:

 * `input_{source}_totn_kgpyr` is the mean amount of TOTN added to the river network from each `{source}` in each kommune.
   
 * `coast_{source}_totn_kgpyr` is the mean amount of TOTN reaching the coast from each `{source}` in each kommune.

Note that the amount of TOTN reaching the coast should always be less than the input due to retention in lakes.

The following `sources` are included:

 * `agriculture-background`: The amount of nutrients that would be lost from agricultural areas if they were not cultivated.
 * `agriculture`: Additional losses from agricultural areas due to farming/land management.
 * `aquaculture`: Nutrient losses from salmon and sea trout farms in seawater.
 * `industry`: Direct discharges from industrial sites (i.e. those not connected to the municipal wastewater network).
 * `lake`: Atmospheric deposition directly to lake surfaces.
 * `large-wastewater`: Discharges from municipal wastewater sites serving more then 50 people.
 * `spredt`: Discharges from small wastewater sites serving 50 people or fewer (septic tanks, minirenseanlegg etc.).
 * `upland`: Natural losses from upland and mountain areas.
 * `urban`: Nutrients in runoff from populated areas.
 * `wood`: Natural losses from forested areas. Note that effects from forest management (clear felling etc.) are included here, even though they are not natural.

In [6]:
# Number of years being considered
n_years = end_yr + 1 - st_yr

# Result cols of interest
cols = ["regine", "year"] + [
    col for col in mod_df.columns if col.startswith("local") and f"_{par}_" in col
]

# Merge data
par_df = pd.merge(mod_df[cols], trans_df[["regine", par]], how="left", on="regine")
par_df = pd.merge(par_df, reg_gdf[["regine", "komnr"]], how="left", on="regine").dropna(
    subset="komnr"
)

# Get local inputs to each regine and estimate amount reaching coast from each source
for col in par_df.columns:
    if col.startswith("local_"):
        in_col = col.replace("local_", "input_")
        out_col = col.replace("local_", "coast_")
        par_df[in_col] = par_df[col]
        del par_df[col]
        par_df[out_col] = par_df[par] * par_df[in_col] / 100

# Sum results for each kommune
par_df = par_df.drop(columns=["regine", "year", par])
par_df = par_df.groupby("komnr").sum()

# Convert totals to averages per year over period of interest
par_df = par_df / n_years
cols = [col.replace("_kg", "_kgpyr") for col in par_df.columns]
par_df.columns = cols

# Tidy
par_df = par_df.query("komnr != '-1'")
par_df = par_df.round(0).astype(int).reset_index()

# Save
par_df.to_excel(xl_path, index=False)

par_df.head()

,komnr,input_agriculture-background_totn_kgpyr,coast_agriculture-background_totn_kgpyr,input_agriculture_totn_kgpyr,coast_agriculture_totn_kgpyr,input_aquaculture_totn_kgpyr,coast_aquaculture_totn_kgpyr,input_industry_totn_kgpyr,coast_industry_totn_kgpyr,input_lake_totn_kgpyr,...,input_large-wastewater_totn_kgpyr,coast_large-wastewater_totn_kgpyr,input_spredt_totn_kgpyr,coast_spredt_totn_kgpyr,input_upland_totn_kgpyr,coast_upland_totn_kgpyr,input_urban_totn_kgpyr,coast_urban_totn_kgpyr,input_wood_totn_kgpyr,coast_wood_totn_kgpyr
0,0301,3661,3404,32953,30632,0,0,0,0,10514,...,397170,397165,3196,2954,2102,1721,139096,132456,96403,77778
1,1101,31645,28764,287396,261196,0,0,3473,3473,26963,...,37824,37805,6607,6092,109412,98809,36701,35873,60732,53745
2,1103,52880,52114,475924,469028,1379294,1379294,5076,5076,2762,...,21843,21843,19283,18681,4437,4383,110208,104568,36294,36101
3,1106,7591,7452,68320,67067,173714,173714,211521,211521,3071,...,150857,150857,1415,1396,8441,8328,53658,51290,10387,9944
4,1108,69908,62676,630498,565293,208494,208494,0,0,31536,...,5924,5923,14252,12635,183077,158241,90898,83722,119823,111278
